In [ ]:
# interpolation dat from hours to minutes

import pandas as pd

data = pd.read_csv("F_PV_instantElectricity.csv").values # shape (length, 2), (timestamp, value)

timestamp = data[:, 0]
value = data[:, 1]

data = dict(zip(timestamp, value))
starttime = 1643666400

timestamp = sorted(data.keys())
import csv
curtime = starttime
index = 0
print(timestamp[index])
results = []
for i in range(34):
  for j in range(660):
    while(True):
      if timestamp[index]<=curtime and curtime<=timestamp[index+1]:
        break
      index += 1

    ans = ((curtime-timestamp[index])*data[timestamp[index+1]] + (timestamp[index+1]-curtime)*data[timestamp[index]]) / (timestamp[index+1]-timestamp[index])
    results.append(ans)
    curtime += 60
  curtime += 13*60*60
import numpy as np
results = np.array(results)
results = results.reshape((34, 660))

with open("F_out.csv", 'w') as csvfile:
  writer = csv.writer(csvfile)
  for i in results:
    writer.writerow(i)

In [ ]:
## test linear regression

import pandas as pd
data_A = pd.read_csv("A_out.csv", header=None).values
sun_A = pd.read_csv("A_sun.csv", header=None).values

data_A = data_A.reshape(-1)
data_A /= data_A.max()
sun_A = sun_A.reshape(-1)
sun_A /= sun_A.max()

data_B = pd.read_csv("B_out.csv", header=None).values
sun_B = pd.read_csv("B_sun.csv", header=None).values

data_B = data_B.reshape(-1)
data_B /= data_B.max()
sun_B = sun_B.reshape(-1)
sun_B /= sun_B.max()

idxx = [14, 15, 16, 17, 20, 21, 25, 26, 30, 31, 32, 33]
idxx_ex = [e for e in range(34) if e not in idxx]
print(idxx_ex)
data_C = pd.read_csv("F_out.csv", header=None).values[idxx, :]
sun_C = pd.read_csv("F_sun.csv", header=None).values[idxx, :]

data_C = data_C.reshape(-1)
data_C /= data_C.max()
sun_C = sun_C.reshape(-1)
sun_C /= sun_C.max()
print(len(data_A))
data = np.concatenate((data_A, data_B, data_C))
sun = np.concatenate((sun_A, sun_B, sun_C))
print(len(data), len(sun))
idx = range(len(data))
from random import sample
valid_len = len(data) // 5 
valid = sample(idx, valid_len)
train = [e for e in idx if e not in valid] 

from sklearn.linear_model import LinearRegression
import numpy as np
model = LinearRegression(fit_intercept=True)
x = sun[train]
y = data[train]
model.fit(x[:, np.newaxis], y)
import matplotlib.pyplot as plt
xfit = np.linspace(0, 1, 1000)
yfit = model.predict(xfit[:, np.newaxis])

plt.scatter(x, y, c='yellow')
plt.plot(xfit, yfit)

X = data[valid]
Y = model.predict(X[:, np.newaxis])
print(np.mean((X-Y)**2))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 18, 19, 22, 23, 24, 27, 28, 29]


In [ ]:
#concat datas
import pandas as pd
data_A = pd.read_csv("A_out.csv", header=None).values
sun_A = pd.read_csv("A_sun.csv", header=None).values


padd_A = padd_A.reshape(-1)
padd_A /= sun_A.max()
print(padd_A.shape)

data_A = data_A.reshape(-1)
data_A /= data_A.max()
sun_A = sun_A.reshape(-1)
sun_A /= sun_A.max()

data_B = pd.read_csv("B_out.csv", header=None).values
sun_B = pd.read_csv("B_sun.csv", header=None).values

padd_B = np.concatenate((padding, sun_B), axis=1)[:, :-1]
padd_B = sun_B.reshape(-1)
padd_B /= sun_B.max()

data_B = data_B.reshape(-1)
data_B /= data_B.max()
sun_B = sun_B.reshape(-1)
sun_B /= sun_B.max()

data_F = pd.read_csv("F_out.csv", header=None).values
sun_F = pd.read_csv("F_sun.csv", header=None).values
idxs = [14, 15, 16, 17, 20, 21, 25, 26, 30, 31, 32, 33]
idxs_ex = [e for e in range(34) if e not in idxs]

data_F_select = data_F[idxs, :].reshape(-1)
data_F_select /= data_F.max()
sun_F_select = sun_F[idxs, :].reshape(-1)
sun_F_select /= sun_F.max()

data_F_ex = data_F[idxs_ex, :].reshape(-1)
data_F_ex /= data_F.max()
sun_F_ex = sun_F[idxs_ex, :].reshape(-1)
sun_F_ex /= sun_F.max()

import csv
with open("Alldata.csv", 'w') as csvfile:
  writer = csv.writer(csvfile)
  for i in range(34):
    for j in range(660):
      writer.writerow([i, j, 1, 0, 0, sun_A[i*660+j], data_A[i*660+j]])
  for i in range(34):
    for j in range(660):
      writer.writerow([i, j, 0, 1, 0, sun_B[i*660+j], data_B[i*660+j]])
  for i, idx in enumerate(idxs):
    for j in range(660):
      writer.writerow([idx, j, 0, 0, 1, sun_F_select[i*660+j], data_F_select[i*660+j]])
  
  # for i, idx in enumerate(idxs_ex):
  #   for j in range(660):
  #     writer.writerow([idx, j, 0, 0, 1, sun_F_ex[i*660+j], data_F_ex[i*660+j]])
